Task
The objective here is to load resnest50 weights on imagenet dataset and fine tune the model to work with "Tiny Imagenet" dataset provided.
Dataset source: https://www.kaggle.com/akash2sharma/tiny-imagenet

In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model,Sequential,Model
from tensorflow.keras.applications import ResNet50,VGG16,resnet50
from tensorflow.keras.layers import Dense, Dropout, Activation, Conv2D, MaxPooling1D,GlobalAveragePooling2D, Embedding, Flatten, Input, add,concatenate
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.optimizers import Adadelta,Adam,RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import cv2

### Dataset Preparation

In [2]:
scale=1./255
train_dir="/home/kashraf/fall2021_coursework/Neural Nets/Assignment3/tiny_image_net/train"
test_dir= "/home/kashraf/fall2021_coursework/Neural Nets/Assignment3/tiny_image_net/train"
train_gen=ImageDataGenerator(rescale=scale)
test_gen=ImageDataGenerator(rescale=scale)
train_generator=train_gen.flow_from_directory(train_dir,target_size=(64,64),batch_size=64)
test_generator=test_gen.flow_from_directory(test_dir,target_size=(64,64),batch_size=64)
print("Train shape:",train_generator.image_shape)
print("Test shape:",test_generator.image_shape)

Found 70000 images belonging to 200 classes.
Found 70000 images belonging to 200 classes.
Train shape: (64, 64, 3)
Test shape: (64, 64, 3)


### Let's load weights of ResNet50 

In [3]:
input_shape=tf.keras.Input(shape=(64,64,3))
res_model=ResNet50(include_top=False,weights='imagenet', input_tensor=input_shape)

## Let's first freeze weights of our base layer
for layer in res_model.layers:
    layer.trainable=False
print("----------------------Resnet50 Archttecure-----------------------------")
print(res_model.summary())



----------------------Resnet50 Archttecure-----------------------------
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
___________________

### 1. Let's add add global average pooling and match with 200 classes

In [4]:
x = res_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(200, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=res_model.input, outputs=predictions)






# model = Sequential()
# model.add(res_model)
# model.add(GlobalAveragePooling2D())
# model.add(Dense(1024, activation='relu'))
# model.add(Dense(200, activation='softmax'))

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [5]:
## 2. Let's freeze conv layers weights and train the model for classification

In [6]:
for layer in model.layers:
    print(layer.name,layer.trainable)

input_1 False
conv1_pad False
conv1_conv False
conv1_bn False
conv1_relu False
pool1_pad False
pool1_pool False
conv2_block1_1_conv False
conv2_block1_1_bn False
conv2_block1_1_relu False
conv2_block1_2_conv False
conv2_block1_2_bn False
conv2_block1_2_relu False
conv2_block1_0_conv False
conv2_block1_3_conv False
conv2_block1_0_bn False
conv2_block1_3_bn False
conv2_block1_add False
conv2_block1_out False
conv2_block2_1_conv False
conv2_block2_1_bn False
conv2_block2_1_relu False
conv2_block2_2_conv False
conv2_block2_2_bn False
conv2_block2_2_relu False
conv2_block2_3_conv False
conv2_block2_3_bn False
conv2_block2_add False
conv2_block2_out False
conv2_block3_1_conv False
conv2_block3_1_bn False
conv2_block3_1_relu False
conv2_block3_2_conv False
conv2_block3_2_bn False
conv2_block3_2_relu False
conv2_block3_3_conv False
conv2_block3_3_bn False
conv2_block3_add False
conv2_block3_out False
conv3_block1_1_conv False
conv3_block1_1_bn False
conv3_block1_1_relu False
conv3_block1_2_con

In [ ]:
for layer in res_model.layers:
    layer.trainable=False
checkpoint = ModelCheckpoint("Resnet_top_TL2.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 3,
                              verbose = 1,
                              min_delta = 0.0001)

# we put our call backs into a callback list
callbacks = [ checkpoint,reduce_lr]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(),
              metrics = ['accuracy'])


epochs = 30

batch_size=train_generator.batch_size
history= model.fit(train_generator,epochs = epochs,
                   callbacks = callbacks,
                   batch_size=batch_size,
                   validation_data = test_generator)



Epoch 1/30
1093/1094 [============================>.] - ETA: 0s - loss: 5.1069 - accuracy: 0.0204
Epoch 00001: val_loss improved from inf to 4.94381, saving model to Resnet_top_TL2.h5
1094/1094 [==============================] - 29s 27ms/step - loss: 5.1068 - accuracy: 0.0204 - val_loss: 4.9438 - val_accuracy: 0.0327 - lr: 0.0010
Epoch 2/30
1092/1094 [============================>.] - ETA: 0s - loss: 4.8729 - accuracy: 0.0395
Epoch 00002: val_loss improved from 4.94381 to 4.80168, saving model to Resnet_top_TL2.h5
1094/1094 [==============================] - 28s 26ms/step - loss: 4.8729 - accuracy: 0.0394 - val_loss: 4.8017 - val_accuracy: 0.0481 - lr: 0.0010
Epoch 3/30
1091/1094 [============================>.] - ETA: 0s - loss: 4.7720 - accuracy: 0.0494
Epoch 00003: val_loss improved from 4.80168 to 4.72039, saving model to Resnet_top_TL2.h5
1094/1094 [==============================] - 28s 26ms/step - loss: 4.7719 - accuracy: 0.0494 - val_loss: 4.7204 - val_accuracy: 0.0535 - lr: 0.0

### Fine tuning convolution layers
Here I will free the first 3 conv layers and train the rest

In [ ]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception Resnet50. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
# for i, layer in enumerate(res_model.layers):
#     print(i, layer.name)

# # we chose to free the first top 3 conv and residual block, i.e. we will freeze
# # the first 80 layers and unfreeze the rest:
for layer in model.layers[:80]:
    layer.trainable = False
for layer in model.layers[80:]:
    layer.trainable = True
# for i, layer in enumerate(model.layers):
#     print(i, layer.trainable)

In [ ]:
checkpoint = ModelCheckpoint("Resnet50_Fine_TL2.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 3,
                              verbose = 1,
                              min_delta = 0.0001)

# we put our call backs into a callback list
callbacks = [ checkpoint,reduce_lr]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(),
              metrics = ['accuracy'])


epochs = 100

batch_size=train_generator.batch_size
history= model.fit(train_generator,epochs = epochs,
                   callbacks = callbacks,
                   batch_size=batch_size,
                   validation_data = test_generator)
